In [1]:
!pip install folium numpy pandas


In [2]:
import folium
import pandas as pd
import numpy as np
from folium.plugins import HeatMap


In [3]:
# Load your dataset
file_path = "geocoded_addresses_google.csv"  # Update if needed
df = pd.read_csv(file_path)

# Drop rows with missing values
df = df.dropna(subset=["latitude", "longitude", "Rental price (per month) BEFORE the increase (put N/A if FMR)", "Rental price (per month) AFTER the increase"])

# Convert rent columns to float (remove "$" and ",")
df["rent_before"] = df["Rental price (per month) BEFORE the increase (put N/A if FMR)"].replace("[\$,]", "", regex=True).astype(float)
df["rent_after"] = df["Rental price (per month) AFTER the increase"].replace("[\$,]", "", regex=True).astype(float)


ValueError: could not convert string to float: '4550 was where it started in 2021 and now its 5670'

In [4]:
# Load the dataset
df = pd.read_csv("geocoded_addresses_google.csv")

# Drop rows with missing coordinates or rent values
df = df.dropna(subset=["latitude", "longitude", "Rental price (per month) BEFORE the increase (put N/A if FMR)", "Rental price (per month) AFTER the increase"])

# Remove "$" and "," and extract only numbers using regex
df["rent_before"] = df["Rental price (per month) BEFORE the increase (put N/A if FMR)"].str.replace("[^\d.]", "", regex=True)
df["rent_after"] = df["Rental price (per month) AFTER the increase"].str.replace("[^\d.]", "", regex=True)

# Convert columns to float, replacing errors with NaN
df["rent_before"] = pd.to_numeric(df["rent_before"], errors="coerce")
df["rent_after"] = pd.to_numeric(df["rent_after"], errors="coerce")

# Drop rows where conversion failed (non-numeric rent values)
df = df.dropna(subset=["rent_before", "rent_after"])

# Confirm the data is now numeric
print(df[["rent_before", "rent_after"]].head())


   rent_before  rent_after
0      12750.0     28000.0
1       5000.0      7000.0
2       8995.0     12000.0
3       9000.0     11000.0
4       8500.0     11500.0


In [5]:
# Calculate percentage increase
df["percent_increase"] = ((df["rent_after"] - df["rent_before"]) / df["rent_before"]) * 100

# Filter only gouging cases (20%+ increase)
df_gouging = df[df["percent_increase"] >= 20]


In [9]:
# Initialize the map centered in Los Angeles
m = folium.Map(location=[34.0522, -118.2437], zoom_start=10, tiles="cartodbpositron")  # Light gray basemap

# Extract latitude, longitude, and severity metric
heat_data = list(zip(df_gouging["latitude"], df_gouging["longitude"], df_gouging["percent_increase"]))

# Add HeatMap layer
HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(m)

from branca.element import Template, MacroElement

# Define the legend's HTML and CSS
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 40px; left: 40px; width: 220px; height: 160px; 
    background-color: white; z-index:9999; font-size:14px;
    border-radius: 8px; padding: 10px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
">
    <b>🔥 Rent Increase Severity</b><br>
    <small>(% increase from previous rent)</small><br>
    <i style="background:#ff0000;width:15px;height:15px;display:inline-block;"></i> Extreme (60%+)<br>
    <i style="background:#ff7300;width:15px;height:15px;display:inline-block;"></i> Very High (40-60%)<br>
    <i style="background:#ffc100;width:15px;height:15px;display:inline-block;"></i> High (20-40%)<br>
    <i style="background:#ffff00;width:15px;height:15px;display:inline-block;"></i> Moderate (10-20%)<br>
    <i styl


# Inject the legend into the map
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().html.add_child(legend)




# Save and display the map
m.save("heatmap_rent_gouging.html")
m


SyntaxError: incomplete input (613957156.py, line 13)

In [10]:


# Initialize the map centered on Los Angeles
m = folium.Map(location=[34.0522, -118.2437], zoom_start=10, tiles="cartodbpositron")  # Light gray basemap

# Extract latitude, longitude, and severity metric
heat_data = list(zip(df_gouging["latitude"], df_gouging["longitude"], df_gouging["percent_increase"]))

# Add HeatMap layer
HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(m)

# Define the legend's HTML and CSS
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 40px; left: 40px; width: 250px; height: 220px; 
    background-color: white; z-index:9999; font-size:14px;
    border-radius: 8px; padding: 10px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
    font-family: Arial, sans-serif;
    line-height: 18px;
">
    <b>🔥 Rent Increase Severity</b><br>
    <small>(% increase from previous rent)</small><br>
    <i style="background:#ff0000;width:15px;height:15px;display:inline-block;"></i> Extreme (60%+)<br>
    <i style="background:#ff7300;width:15px;height:15px;display:inline-block;"></i> Very High (40-60%)<br>
    <i style="background:#ffc100;width:15px;height:15px;display:inline-block;"></i> High (20-40%)<br>
    <i style="background:#ffff00;width:15px;height:15px;display:inline-block;"></i> Moderate (10-20%)<br>
    <i style="background:#00ff00;width:15px;height:15px;display:inline-block;"></i> Low (<10%)<br>
    <hr style="margin-top: 5px; margin-bottom: 5px;">
    <i><small>This heatmap visualizes the severity of rent increases across Los Angeles County following the wildfires that occurred in January 2025. The map uses geocoded rental listings scraped from Zillow and AirBnB by The Rent Brigade. Pre-wildfire and post-wildfire rental prices were used to calculate the percent increase of each listing. Redder parts of the heatmap have a higher concentration of listings that increased significantly in price following the fires.</small></i><br>
    <i><small>The percentage increase for each listing is calculated as:</small></i>
    <br>
    <i><small>Percent Increase = ((Rent After - Rent Before) / Rent Before) × 100</small></i>
</div>
{% endmacro %}
"""

# Inject the legend into the map
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().html.add_child(legend)

# Save and display the updated heatmap
m.save("heatmap_rent_gouging.html")
m


In [12]:



# Initialize the map centered on Los Angeles
m = folium.Map(location=[34.0522, -118.2437], zoom_start=10, tiles="cartodbpositron")  # Light gray basemap

# Extract latitude and longitude for all addresses
heat_data = list(zip(df["latitude"], df["longitude"]))

# Add HeatMap layer
HeatMap(heat_data, radius=12, blur=8, max_zoom=1).add_to(m)

# Define the legend's HTML and CSS
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 40px; left: 40px; width: 250px; height: 140px; 
    background-color: white; z-index:9999; font-size:14px;
    border-radius: 8px; padding: 10px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
    font-family: Arial, sans-serif;
    line-height: 18px;
">
    <b>📍 Rental Listings Density</b><br>
    <small>Heatmap shows where rental listings are concentrated.</small><br>
    <i style="background:#ff0000;width:15px;height:15px;display:inline-block;"></i> Very High Density<br>
    <i style="background:#ff7300;width:15px;height:15px;display:inline-block;"></i> High Density<br>
    <i style="background:#ffc100;width:15px;height:15px;display:inline-block;"></i> Moderate Density<br>
    <i style="background:#ffff00;width:15px;height:15px;display:inline-block;"></i> Low Density<br>
    <i style="background:#00ff00;width:15px;height:15px;display:inline-block;"></i> Very Low Density<br>
</div>
{% endmacro %}
"""

# Inject the legend into the map
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().html.add_child(legend)

# Save and display the heatmap
m.save("heatmap_all_addresses.html")
m


In [21]:
# Legend still isn't big enough LOL let's try again. 


# Initialize the map centered on Los Angeles
m = folium.Map(location=[34.0522, -118.2437], zoom_start=10, tiles="cartodbpositron")  # Light gray basemap

# Extract latitude and longitude for all addresses
heat_data = list(zip(df["latitude"], df["longitude"]))

# Add HeatMap layer
HeatMap(heat_data, radius=12, blur=8, max_zoom=1).add_to(m)

# Define the legend's HTML and CSS with an expanded background and updated description
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 40px; left: 40px; width: 270px; height: 250px;  /* Further increased height */
    background-color: white; z-index:9999; font-size:14px;
    border-radius: 8px; padding: 15px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
    font-family: Arial, sans-serif;
">
    <b>📍 Rental Price Gouging Density in Los Angeles</b><br>
    <small>Heatmap shows where rentals that increased by 10% or more in price following the January 2025 wildfires are concentrated.</small><br><br>
    <i style="background:#ff0000;width:15px;height:15px;display:inline-block; margin-right: 8px;"></i> Very High Density<br>
    <i style="background:#ff7300;width:15px;height:15px;display:inline-block; margin-right: 8px;"></i> High Density<br>
    <i style="background:#ffc100;width:15px;height:15px;display:inline-block; margin-right: 8px;"></i> Moderate Density<br>
    <i style="background:#ffff00;width:15px;height:15px;display:inline-block; margin-right: 8px;"></i> Low Density<br>
    <i style="background:#00ff00;width:15px;height:15px;display:inline-block; margin-right: 8px;"></i> Very Low Density<br>
</div>
{% endmacro %}
"""

# Inject the legend into the map
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().html.add_child(legend)

# Save and display the heatmap
m.save("heatmap_all_addresses.html")
m
